In [8]:
import time
from src.method_selector import MlMethodSelector, ClassicMethodSelector
from src.measurement_provider import MeasurementProvider
from src.serializer import SenMLCBORSerializer, SenMLJSONSerializer
from src.signal_generator import SignalGenerator
from sys import getsizeof
from src.metric import FeatureMetricEnum, SimilarityMetricEnum
from src.data_type import Measurement
import pandas as pd
import matplotlib as plt
from src.metric import SimilarityMetricEnum
plt.rcParams["figure.dpi"] = 100

In [ ]:
for datasize in [i * 100 for i in range(11)]:
  dataset = []
  getsizeof(SenMLCBORSerializer.serialize(dataset, '/72/', '1/2', metrics))
  getsizeof(SenMLJSONSerializer.serialize(dataset, '/72/', '1/2', metrics))

In [ ]:
pd.DataFrame.from_dict()